# Imports

In [1]:
import numpy as np
from pprint import pprint as pp
import re #Used to read chunks from the data files - clusters, vmat, etec.
import math
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import random
from IPython.display import clear_output

from scipy.optimize import minimize, basinhopping
from scipy.optimize import SR1, BFGS
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint

from ase import Atoms
from ase.units import kB
from ase.visualize import view
from ase.build import bulk
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.vasp import write_vasp

import sys
import subprocess
import os

pattern1 = re.compile("\n\n\n")
pattern2 = re.compile("\n\n")
np.set_printoptions(suppress=True,precision=2)

# Read all necessary files

## Read clusters.out

In [2]:
# Read clusters.out
clusters = {}

with open('clusters.out','r') as fclusters:
    temp_clusters = fclusters.read().split('\n\n') #Read blocks separated by 1 empty line

for idx, cluster in enumerate(temp_clusters):
    if cluster == '': #Check for spurious empty blocks
        continue 
    line = cluster.split('\n') #If not empty split by lines
    multiplicity = int(line[0]) #1st line
    length = float(line[1]) #largest distance between two atoms
    num_points = int(line[2]) #type of cluster
    clusters[idx] = {'mult':multiplicity, 'length':length, 'type':num_points}

## Read config.out

In [3]:
# Read config.out
configs = {}

fconfig = open('config.out','r')
_ = next(fconfig) #Ignore first line

temp_config = fconfig.read()#.split('\n\n')
temp_config = pattern1.split(temp_config) #split lines separated by 2 empty lines

for idx, config in enumerate(temp_config):
    if config == '': #Check for spurious empty blocks
        continue
    num_points = int(config[0]) #number of subclusters
    config = pattern2.split(config[2:]) #now split individual subclusters separated by 1 blank line
    min_coords = []
    for _ in range(num_points):
        min_coords.append(config[_].split('\n')[0])
    configs[idx] = {'subclus': list(map(int,min_coords)), 'num_of_subclus': len(min_coords)}

## Read Kikuchi-Baker coefficients

In [4]:
# Read kb.out

kb = {}
fkb = open('kb.out','r')
_ = next(fkb) #ignore first line

temp_kb = fkb.read()
temp_kb = temp_kb.split('\n') #split file linewise

for idx, kbcoeff in enumerate(temp_kb):
    if kbcoeff == '': #check for spurious empty blocks
        continue
    kb[idx] = float(kbcoeff)

fkb.close()

## Read coefficients of subclusters

In [5]:
# Read configcoeff.out
configcoef = {}

with open('configcoef.out','r') as fsubmult:
    _ = next(fsubmult) #ignore first line
    temp_submult = fsubmult.read() 
    temp_submult = pattern2.split(temp_submult) #split lines into blocks separated by 2 empty lines
    
for idx, submult in enumerate(temp_submult):
    submult = submult.split('\n') #split into number of subclusters 
    while("" in submult) :
        submult.remove("") #remove empty blocks
    configcoef[idx] = list(map(float,submult[1:])) #also ignore 1st line of each block

## Read V-Matrix

In [6]:
# Read vmat.out
vmat = {}
with open('vmat.out') as fvmat:
    _ = next(fvmat) #ignore first lie
    temp_vmat = fvmat.read()
    temp_vmat = pattern2.split(temp_vmat) #split by 2 empty lines i.e. maxclusters
    
    while("" in temp_vmat):
        temp_vmat.remove("") #remove empty blocks
    
    for clus_idx, mat in enumerate(temp_vmat):
        mat = mat.split('\n') #split by 1 empty line i.e. subclusters
        mat_float = np.empty(list(map(int, mat[0].split(' '))))
        for idx, row in enumerate(mat[1:]): #ignore first line
            mat_float[idx] = list(map(float,row.split(' ')[:-1]))
        
        vmat[clus_idx] = mat_float

## Read ECI

In [7]:
# Read eci
eci = {}

with open('eci.out') as feci:
    _ = next(feci) #Ignore first line
    temp_eci = feci.read()
    temp_eci = temp_eci.split('\n') #split by line

for idx, eci_val in enumerate(temp_eci):
    if eci_val == '':
        continue
    eci[idx] = float(eci_val)

In [8]:
#In this notebook ECI's are declared manually
eci = {0: 0, 1: 0, 2: 0.1, 3: 0.0, 4: 0, 5: 0}

In [9]:
eci

{0: 0, 1: 0, 2: 0.1, 3: 0.0, 4: 0, 5: 0}

In [10]:
corrs1 = np.array([1., 1., 1., 1., 1., 1.]) #Pure B
corrs0 = np.array([1., -1., 1., 1., -1., 1.]) #Pure A
corrssqs = np.array([1.    , 0.0   , 0.25,0.25  , 0.125 , 0.0625]) # AB - sqs
corrsrand = np.array([1.   , *np.random.uniform(-1, 1, 5)]) 
T = 1/kB
print(corrsrand)

[ 1.   -0.52 -0.13 -0.42 -0.9  -0.57]


# Setting up Log Absolute functions

## Set up F

In [11]:
def F(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    F = H + kB*T*SUM(rho * log(rho))
    """
    
    def get_corrsum(vmat,corrs):
        assert len(vmat) == len(corrs)
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return 0
            #corrsum = np.finfo(float).tiny

        return corrsum * math.log(np.abs(corrsum))
    
    def per_cluster_sum(corrs,vmat,configcoef):
        config_sum = np.sum([coef * get_corrsum(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)
                            ])
                      
        return config_sum
    
    H = np.sum([cluster['mult']*eci[cluster_idx]*corrs[cluster_idx] 
                for cluster_idx, cluster in clusters.items()
               ])
    
    S = np.sum([kb[cluster_idx]*per_cluster_sum(corrs,
                                                vmat[cluster_idx],
                                                configcoef[cluster_idx],)
                for cluster_idx in clusters.keys()
               ])
    
    return H + kB*T*S

In [12]:
f0 = F(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f1 = F(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
fsqs = F(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
frand = F(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f0:.2f} -- Corrs 1: {f1:.2f} -- Corrs Rand: {frand:.2f} -- Corrs SQS: {fsqs:.2f}")

Corrs 0: 0.40 -- Corrs 1: 0.40 -- Corrs Rand: -2.22 -- Corrs SQS: -2.53


## Set up F Jacobian

In [13]:
def F_jacobian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [dF/dcorr0, dF/dcorr1, ...]
    """
    
    def get_kth_elem_jac(corrs, vmat, kb, clusters, configs, configcoef,corr_idx):
        
        dS_k = np.sum([kb[cluster_idx]*per_cluster_sum_jac(corrs,
                                                           vmat[cluster_idx],
                                                           configcoef[cluster_idx],
                                                           corr_idx,) 
                       for cluster_idx in clusters.keys()
                      ])
        return dS_k
    
    def get_corrsum_jac(vmat,corrs):
        assert len(vmat) == len(corrs)
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return np.NINF
            #corrsum = np.finfo(float).tiny

        return 1 + math.log(np.abs(corrsum))

    def per_cluster_sum_jac(corrs,vmat,configcoef,corr_idx):
        
        config_sum = np.sum([coef * vmat[config_idx][corr_idx] * get_corrsum_jac(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)
                            ])

        
        return config_sum
    
    dH = np.array([cluster['mult']*eci[cluster_idx] for cluster_idx, cluster in clusters.items()])
    
    dS = np.array([get_kth_elem_jac(corrs, vmat, kb, clusters, configs, configcoef,corr_idx) for corr_idx, _ in enumerate(corrs)])
    
    return dH + kB*T*dS

In [14]:
f_jaco0 = F_jacobian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_jaco1 = F_jacobian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_jacosqs = F_jacobian(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
f_jacorand = F_jacobian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0: {f_jaco0} \nCorrs 1: {f_jaco1} \nCorrs Rand {f_jacorand} \nCorrs SQS {f_jacosqs}")

Corrs 0: [nan nan nan nan nan nan] 
Corrs 1: [nan nan nan nan nan nan] 
Corrs Rand [-1.88 -0.83  1.37 -0.46 -0.16 -0.14] 
Corrs SQS [-1.9  -0.1   0.97  0.41  0.23 -0.05]


/home/sayan/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-13-873df088579e>:38: RuntimeWarning: invalid value encountered in double_scalars
  config_sum = np.sum([coef * vmat[config_idx][corr_idx] * get_corrsum_jac(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)


## Set up Hessian

In [15]:
def F_hessian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [[d^2F/dcorr0 dcorr0, d^2F/dcorr0 dcorr1, ..., d^2F/dcorr0 dcorrn],
     [d^2F/dcorr1 dcorr0, d^2F/dcorr1 dcorr1, ..., d^2F/dcorr1 dcorrn],
     .
     .
     .
     [d^2F/dcorrn dcorr0, d^2F/dcorrn dcorr1, ..., d^2F/dcorrn dcorrn],
    ]
    """
    
    def get_corrsum_hess(vmat,corrs):
        assert len(vmat) == len(corrs)
        
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return 1/np.PINF
            #corrsum = np.finfo(float).tiny

        return corrsum
    
    def get_config_val(corrs,vmat,configcoef,corr_idx_1,corr_idx_2):
        
        config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs) 
                             for config_idx, coef in enumerate(configcoef)
                            ])
        
        return config_val
    
    def get_hessian_elem(corrs, vmat, kb, clusters, configs, configcoef,T,eci,corr_idx_1,corr_idx_2):
        
        hess_elem = np.sum([kb[cluster_idx] * get_config_val(corrs,
                                                             vmat[cluster_idx],
                                                             configcoef[cluster_idx],
                                                             corr_idx_1,
                                                             corr_idx_2
                                                            ) 
                            for cluster_idx in clusters.keys()
                           ])
        return hess_elem
    
    d2F = np.empty([len(corrs),len(corrs)])
    
    d2F = np.array([[get_hessian_elem(corrs, vmat, kb, clusters, configs, configcoef, T, eci, corr_idx_1, corr_idx_2) for corr_idx_2, _ in enumerate(corrs)] for corr_idx_1, _ in enumerate(corrs)])
    
    return d2F

In [16]:
f_hess0 = F_hessian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_hess1 = F_hessian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_hessrand = F_hessian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
f_hesssqs = F_hessian(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
print(f"Corrs 0:\n {f_hess0} \n Corrs 1:\n {f_hess1}\n Corrs Rand:\n {f_hessrand} \n Corrs SQS:\n {f_hesssqs}")

Corrs 0:
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] 
 Corrs 1:
 [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
 Corrs Rand:
 [[-1.46 -5.33 -5.08  0.91  0.15  0.79]
 [-5.33 -7.38 -3.37 -1.22 -0.82  0.31]
 [-5.08 -3.37 13.83 -7.27  0.18 -4.02]
 [ 0.91 -1.22 -7.27  4.05 -0.14  1.63]
 [ 0.15 -0.82  0.18 -0.14  0.99 -0.5 ]
 [ 0.79  0.31 -4.02  1.63 -0.5   1.63]] 
 Corrs SQS:
 [[ 1.25  0.13 -0.55 -0.41 -0.16  0.14]
 [ 0.13  2.99  0.02  0.01 -0.89 -0.35]
 [-0.55  0.02  3.49 -0.97 -0.25 -0.69]
 [-0.41  0.01 -0.97  2.76 -0.13 -0.34]
 [-0.16 -0.89 -0.25 -0.13  1.94  0.27]
 [ 0.14 -0.35 -0.69 -0.34  0.27  1.29]]


<ipython-input-15-d4a292fd34bd>:35: RuntimeWarning: divide by zero encountered in double_scalars
  config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs)
<ipython-input-15-d4a292fd34bd>:35: RuntimeWarning: invalid value encountered in double_scalars
  config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs)


# Constraints

In [70]:
def constraint_rhos_sum(corrs, vmat, clusters, configcoef,):
    """
    Constraints the sum of each rho. As of now, it's done in a weird way, where the total sum of the array:
    [1 - sum(rho), .... ] is constrained to sum to 0. This along with the constraint that each rho is between
    0 and 1, seems to make it work. I think that by the this might be a redundant constraint as well.
    """
    rho_sum = []

    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        rho_sum.append(np.sum(configcoef[cluster_idx]*rho))
    
    return np.sum(1 - np.array(rho_sum))

# def constrain_rhos_sum(corrs,vmat_sub,configcoef_sub):
    
#     rho_sub = np.matmul(vmat_sub,corrs)
#     assert len(rho) == len(configcoef_sub)
    
#     return np.sum(1 - np.inner(configcoef_sub,rho_sub))
    

def constraint_singlet(corrs,FIXED_CORR_1):
    """
    constrains the 1-pt correlation:
    corrs[1] = FIXED_CORR_1
    """
    return corrs[1] - FIXED_CORR_1   

def constraint_zero(corrs):
    """
    constrains the 1-pt correlation:
    corrs[0] = 1
    """
    return 1 - corrs[0] 

class MyBounds:
    """
    Class to constrain the trial correlations of Basin Hopping
    """
    def __init__(self, xmax=[1]*6, xmin=[-1]*6):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))

        return tmax and tmin

In [85]:
np.linspace(0, 2000, num=21)

array([   0.,  100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,
        900., 1000., 1100., 1200., 1300., 1400., 1500., 1600., 1700.,
       1800., 1900., 2000.])

In [79]:
np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,21)

array([-1. , -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1,  0. ,
        0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])

In [ ]:
#{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,]},

# Uniform Sampling

In [58]:
results_uniform = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])
NUM_TRIALS = 10

for temp in [0,100,500,1000,3000]:#tqdm(np.linspace(0, 2000, num=20)):
    for x in tqdm(np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,9)):
        FIXED_CORR_1 = x
        
        MIN_RES_VAL = 1e5 #random large number
        rho_pair = np.array([None,None])
        #corrs0 = np.array([1, *np.random.uniform(-1, 1, 2)])
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():
            
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)]
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 5000,
                   'xtol'    : 1e-9,
                   'initial_constr_penalty' : 10,
                  }
        
        for _ in tqdm(range(NUM_TRIALS)):
            
            corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
            res = minimize(F,
                           corrs0,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci),
                           options=options,
                           #jac='3-point',
                           #hess=BFGS(),
                           jac=F_jacobian,
                           hess=F_hessian,
                           constraints=[*linear_constraints, 
                                        {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                        {'fun': constraint_zero, 'type':'eq',},
                                        #{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,]},
                                       ],
                           bounds=bounds_corrs,
                          )
            
            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for x:{x}, T:{temp} fun: {MIN_RES_VAL}")
        
            
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_uniform = results_uniform.append({'T' : temp, 
                                           '1-point_corr' : x, 
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                          }, 
                                          ignore_index = True
                                         )
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
results_uniform.to_pickle(f'phaseDiag_{now}_uniform.pickle')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning:

Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.

/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.



Found new minimum for x:-0.9999999999999998, T:0 fun: 0.3999999888292458
Found new minimum for x:-0.9999999999999998, T:0 fun: 0.3999999862329245
Found new minimum for x:-0.9999999999999998, T:0 fun: 0.39999998537663684
Found new minimum for x:-0.9999999999999998, T:0 fun: 0.3999999831831035


  0%|          | 0/10 [00:00<?, ?it/s]

...

Found new minimum for x:0.4999999999999998, T:3000 fun: -0.3670200191317903
Found new minimum for x:0.4999999999999998, T:3000 fun: -0.4836140672876081


  0%|          | 0/10 [00:00<?, ?it/s]

Found new minimum for x:0.7499999999999998, T:3000 fun: -0.12104714156824148
Found new minimum for x:0.7499999999999998, T:3000 fun: -0.1254229439976237
Found new minimum for x:0.7499999999999998, T:3000 fun: -0.12756776696307706
Found new minimum for x:0.7499999999999998, T:3000 fun: -0.13124717844704278
Found new minimum for x:0.7499999999999998, T:3000 fun: -0.1354572464880901


  0%|          | 0/10 [00:00<?, ?it/s]

Found new minimum for x:0.9999999999999998, T:3000 fun: 0.3999999128433328
Found new minimum for x:0.9999999999999998, T:3000 fun: 0.3999998341141116
Found new minimum for x:0.9999999999999998, T:3000 fun: 0.3999998199879255


## Plot Phase Diagram

In [60]:
fig = go.Figure()

for T in results_uniform['T'].unique():
    fig.add_trace(go.Scatter(x = results_uniform[results_uniform['T'] == T]['1-point_corr'],
                             y = results_uniform[results_uniform['T'] == T]['F'],
                             mode='markers+lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

## Plot Pair Correlation

In [61]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = results_uniform[results_uniform['1-point_corr'] == 0.0]['T'],
                         y = results_uniform[results_uniform['1-point_corr'] == 0.0]['corrs'].str[2],
                         mode='markers+lines',
                        )
             )

fig.update_layout(
    title="T vs 2-point Corr",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

In [128]:
np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,19)

array([-1.  , -0.89, -0.78, -0.67, -0.56, -0.44, -0.33, -0.22, -0.11,
        0.  ,  0.11,  0.22,  0.33,  0.44,  0.56,  0.67,  0.78,  0.89,
        1.  ])

# Basin Hopping

In [ ]:
results_basinhopping = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])

now = datetime.now()
now = now.strftime("%d-%m-%H-%M")

for temp in tqdm(np.linspace(0, 2000, num=21)):
    for x in tqdm(np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,21)):

        FIXED_CORR_1 = x

        rho_pair = np.array([None,None])
        
        #Make a first guess
        corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
        linear_constraints = []

        #Linear Constraint for each rho to be between 0 and 1
        for cluster_idx, _ in clusters.items():
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        #Set bounds for the local minimisation 
        #limit correlations to [1, 1-pt, [-1,1], [-1,1], ...]
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)]
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 1000,
                   'xtol'    : 1e-6,
                   'initial_constr_penalty' : 10,
                  }
        
        minimizer_kwargs = {'args':(vmat, kb, clusters, configs, configcoef,temp,eci),
                            'method': 'trust-constr',
                            'options': options,
                            'jac': F_jacobian, 'hess': F_hessian,
                            'constraints' : [*linear_constraints, 
                                             {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                             {'fun': constraint_zero, 'type':'eq',},
                                             #{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,]},
                                            ],
                            'bounds': bounds_corrs,
                           }
        
        mybounds = MyBounds(xmax=[1, FIXED_CORR_1,*[1]*(len(clusters)-2)], xmin=[1, FIXED_CORR_1,*[-1]*(len(clusters)-2)])
        
        res = basinhopping(F, 
                           corrs0, #first guess
                           niter=200, #total num of iterations
                           T=1.0, #temp for Metropolis MC trial search
                           stepsize=5,
                           minimizer_kwargs=minimizer_kwargs,
                           niter_success=5, #num iters to exit after no new minima found 
                           interval=5, #num iters to change step size
                           disp=True,
                           accept_test=mybounds,
                           seed=42,
                           #callback=basin_hopping_callback
                          )

        #Code to extract rhos and check if they sum them to 1 for sanity. Not used.
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_basinhopping = results_basinhopping.append({'T' : temp, 
                                                     '1-point_corr' : x, 
                                                     'F' : res.fun, 
                                                     'corrs': res.x,
                                                    }, 
                                                    ignore_index = True
                                                   )

#save results
results_basinhopping.to_pickle(f'phaseDiag_{now}_basinhopping.pickle')

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_trustregion_constr/projections.py:181: UserWarning:

Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.

/home/sayan/.local/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.



basinhopping step 0: f 0.4
basinhopping step 1: f 0.4 trial_f 0.4 accepted 0  lowest_f 0.4
basinhopping step 2: f 0.4 trial_f 0.4 accepted 0  lowest_f 0.4
basinhopping step 3: f 0.4 trial_f 0.4 accepted 0  lowest_f 0.4
basinhopping step 4: f 0.4 trial_f 0.4 accepted 0  lowest_f 0.4
adaptive stepsize: acceptance rate 0.000000 target 0.500000 new stepsize 4.5 old stepsize 5
basinhopping step 5: f 0.4 trial_f 0.4 accepted 0  lowest_f 0.4
basinhopping step 6: f 0.4 trial_f 0.4 accepted 0  lowest_f 0.4
basinhopping step 0: f 0.320015
basinhopping step 1: f 0.36533 trial_f 0.36533 accepted 1  lowest_f 0.320015
basinhopping step 2: f 0.320023 trial_f 0.320023 accepted 1  lowest_f 0.320015
basinhopping step 3: f 0.320034 trial_f 0.320034 accepted 1  lowest_f 0.320015
basinhopping step 4: f 0.320015 trial_f 0.320015 accepted 1  lowest_f 0.320015
adaptive stepsize: acceptance rate 0.800000 target 0.500000 new stepsize 5.55556 old stepsize 5
basinhopping step 5: f 0.320015 trial_f 0.320015 accept

basinhopping step 8: f 0.00111147 trial_f 0.373865 accepted 0  lowest_f 1.53126e-05


## Plot Phase Diagram

In [82]:
fig = go.Figure()

for T in results_basinhopping['T'].unique():
    fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['T'] == T]['1-point_corr'],
                             y = results_basinhopping[results_basinhopping['T'] == T]['F'],
                             mode='lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

## Plot Pair Correlation



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['T'],
                         y = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['corrs'].str[2],
                         mode='markers+lines',
                        )
             )

fig.update_layout(
    title="T vs 2-point Corr",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()